# **Analysis of Optimization Results**

In [1]:
import sys
sys.path
sys.path.append('c:\\Users\\3leso\\Documents\\Elena\\Uni\\MasterThesis\\MasterThesis_GitHub\\code\\faultana')

In [2]:
import pandas as pd

In [3]:
from script_faultana_functions import measures

In [4]:
subreddit_here = 'politics'

### Import data

In [ ]:
optim_data = pd.read_csv(f"output/optimization_data_{subreddit_here}_frac1.csv")
optim_result = pd.read_csv(f"output/result_optimization_data_{subreddit_here}_frac1.csv")
optim_result.drop(columns = ["Unnamed: 0"], inplace = True)


In [16]:
optim_result
#optim_result

,Community,Node
0,1,0
1,0,1
2,1,2
3,1,3
4,1,4
...,...,...
3045,0,3045
3046,0,3046
3047,0,3047
3048,1,3048


## **Calculate Metrics**
Cohesiveness and Divisiveness, frustrated edges

In [17]:
# extract unique users and count 

user_samples = set(list(optim_data['user_1_id']) + list(optim_data['user_2_id']))
user_counts = len(user_samples) # 3050

- within cluster
    - negative edge --> 1
    - positive edge --> 3

- between cluster
    - negative edge --> -3
    - positive edge --> -1


In [25]:

# balance_vals = {}
# norm_coh_vals = {}
# norm_div_vals= {}
# coh_vals = {}
# div_vals = {}
# frustrated = {}

#for subreddit, net_data in net_samples.items():

print('Network with', user_counts, 'nodes')
solution = optim_result
edgelist = optim_data 

# extract node ids for both communities
users1 = solution[solution["Community"] == 1 ]['Node'].values
users2 = solution[solution["Community"] == 0 ]['Node'].values
size1 = len(users1)
size2 = len(users2)
total_users = size1 + size2
# print member-shares 
print(size1/total_users, size2/total_users)

# create dictionary user_id - community
dict_users = {}
for u in users1:
    dict_users[u] = 0
for u2 in users2:
    dict_users[u2] = 1

# reverse the dictionary: community - user_id
dict_inv = {v: k for k, v in dict_users.items()}
# add community-belonging for each node using the dictionary
edgelist['source_comm'] = edgelist['source']
edgelist['target_comm'] = edgelist['target']
edgelist.replace({'source_comm': dict_users, "target_comm" : dict_users}, inplace=True)

# do translation of within or between community and pos or neg to type
# within cluster
    ## negative edge --> 1
    ## positive edge --> 3

# between cluster
    ## negative edge --> -3
    ## positive edge --> -1
edgelist['type_of_edge'] = 2*(2*edgelist['source_comm']-1)*(2*edgelist['target_comm']-1)+((edgelist['sign']))
prop_negatives = edgelist[edgelist['sign'] == -1].shape[0]/edgelist.shape[0]

print(measures(edgelist['type_of_edge']))

balance, total, frustration_negative, frustration_positive, total_negative, internal, coh, div = measures(edgelist['type_of_edge'])
print('Coh', coh)
print('Div', div)

norm_div = div - prop_negatives
norm_coh = coh - (1-prop_negatives)

print('Norm Coh', norm_coh)
print('Norm Div', norm_div)

# save values
balance_vals = balance
frustrated = frustration_negative+frustration_positive
norm_coh_vals = norm_coh
norm_div_vals = norm_div
coh_vals = coh
div_vals = div

Network with 3050 nodes
0.4465573770491803 0.5534426229508197
(0.6632064590542099, 6069, 539, 483, 2661, 3464, 0.8443995381062356, 0.8145873320537428)
Coh 0.8443995381062356
Div 0.8145873320537428
Norm Coh 0.28285727414182626
Norm Div 0.3761295960181521


Cohesiveness: proportion of internal edges that are positive

Divisiveness: proportion of external edges that are negative 

In [26]:
# weight_dict = {1: 1, -1 :0}
# net_samples['brexit']['weight'] = net_samples['brexit']['sign'].map(weight_dict)

optim_data

,source,target,user_1_id,user_2_id,sign,source_comm,target_comm,type_of_edge
0,1351,419,g38s2x0,kdms7zz,1,0,1,-1
1,1181,2389,gi293l3,g1gufzx,1,1,1,3
2,1181,558,gi293l3,ho67956,1,1,1,3
3,1263,107,c5ehsy5,c6nqppj,1,1,1,3
4,947,558,jmx0zuu,ho67956,1,1,1,3
...,...,...,...,...,...,...,...,...
6064,143,252,k5o2dji,iocsdo5,1,1,1,3
6065,491,1022,kbtcqmo,kexhtol,-1,1,1,1
6066,491,1447,kbtcqmo,kdx8ati,-1,1,0,-3
6067,2526,3039,h2fjv4q,ixceh56,1,1,1,3


In [27]:
optim_data.to_csv(f"output/{subreddit_here}_edgelist.csv", index = False)